<a href="https://colab.research.google.com/github/gomezalliyahdennisse/CCDEPLRL_EXERCISES_COM211/blob/main/Exercise4_CCDEPLRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 4

In [154]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [155]:
import numpy as np
import pandas as pd

dataset = pd.read_json('reviews.json')


In [156]:
dataset.head()

,review,rating
0,sir okay armygreen shorts nice,5
1,di pareha yong mga size nila may sobrang liit ...,5
2,super worth it ang ganda Sombra grabi order na...,5
3,ganda po salamat,5
4,maayos pagkadeliver maganda den sya,5


In [157]:
mask = (dataset['rating'] > 0 ) & (dataset['rating'] < 4)
column_name = 'rating'
dataset.loc[mask, column_name] = 0

mask = (dataset['rating'] > 3 ) & (dataset['rating'] < 6)
column_name = 'rating'
dataset.loc[mask, column_name] = 1


## 1. Tokenize the data

In [158]:
# answer here

# Get the reviews from the text column
reviews = dataset['review'].tolist()

tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(reviews)

word_index = tokenizer.word_index
print(len(word_index))
print(word_index)

3537
{'<OOV>': 1, 'na': 2, 'the': 3, 'ko': 4, 'i': 5, 'ang': 6, 'sa': 7, 'yung': 8, 'and': 9, 'ng': 10, 'lang': 11, 'it': 12, 'is': 13, 'po': 14, 'naman': 15, 'at': 16, 'seller': 17, 'to': 18, 'you': 19, 'order': 20, 'for': 21, 'not': 22, 'pero': 23, 'maganda': 24, 'pa': 25, 'hindi': 26, 'but': 27, 'sya': 28, 'good': 29, 'item': 30, 'thank': 31, 'a': 32, 'dumating': 33, 'sana': 34, 'so': 35, 'ung': 36, 'product': 37, 'my': 38, 'color': 39, 'this': 40, 'size': 41, 'quality': 42, 'was': 43, 'delivery': 44, 'di': 45, 'ok': 46, 'din': 47, 'may': 48, 'of': 49, 'its': 50, 'ako': 51, 'in': 52, 'very': 53, 'nyo': 54, 'lng': 55, 'price': 56, 'mga': 57, 'ganda': 58, 'time': 59, 'that': 60, 'working': 61, 'kasi': 62, 'black': 63, 'me': 64, 'buy': 65, 'items': 66, 'siya': 67, 'sobrang': 68, 'will': 69, 'white': 70, 'well': 71, 'ordered': 72, 'with': 73, 'kulay': 74, 'kaso': 75, '2': 76, 'as': 77, 'nice': 78, 'fast': 79, 'manipis': 80, 'okay': 81, '1': 82, 'lazada': 83, 'kung': 84, 'nya': 85, 'on':

## 2. Sequence the data

In [159]:
vocab_size = 3500
embedding_dim = 256
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

sequences = tokenizer.texts_to_sequences(dataset['review'])

## 3. Pad the data

In [160]:
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

## 4. Train a sentiment model

In [161]:
model = tf.keras.Sequential([
   tf.keras.layers.Embedding(len(word_index) + 1, 200, input_length=max_length),
   tf.keras.layers.GlobalAveragePooling1D(),
   tf.keras.layers.Dense(64, activation='relu'),
   tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, np.array(dataset['rating']), epochs=30, validation_split=0.2)



Epoch 1/30
25/25 [==============================] - 3s 108ms/step - loss: 0.6868 - accuracy: 0.5587 - val_loss: 0.7632 - val_accuracy: 0.2139
Epoch 2/30
25/25 [==============================] - 3s 121ms/step - loss: 0.6824 - accuracy: 0.5688 - val_loss: 0.7950 - val_accuracy: 0.2139
Epoch 3/30
25/25 [==============================] - 2s 79ms/step - loss: 0.6796 - accuracy: 0.5688 - val_loss: 0.7641 - val_accuracy: 0.2139
Epoch 4/30
25/25 [==============================] - 1s 59ms/step - loss: 0.6731 - accuracy: 0.5700 - val_loss: 0.7733 - val_accuracy: 0.2139
Epoch 5/30
25/25 [==============================] - 1s 55ms/step - loss: 0.6573 - accuracy: 0.5788 - val_loss: 0.7425 - val_accuracy: 0.2488
Epoch 6/30
25/25 [==============================] - 1s 52ms/step - loss: 0.6223 - accuracy: 0.6587 - val_loss: 0.6511 - val_accuracy: 0.7015
Epoch 7/30
25/25 [==============================] - 1s 60ms/step - loss: 0.5612 - accuracy: 0.7775 - val_loss: 0.6256 - val_accuracy: 0.6816
Epoch 8/30


## Get files for visualing the network

In [162]:
# answer here
model.save('tagalogreviews_sentiment_model.h5')
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(3538, 200)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## 5. Predict sentiment with new reviews

In [164]:
# Use the model to predict a review
fake_reviews = ['Ang panget!',
                'Hindi na ko bibili ulit, kulang ang pinadala niyo!',
                'Sobrang ganda po, salamat',
                'Maliliit sizes, hindi kasya sakin kahit large na inorder ko.' ,
                'Ang ganda po ng quality',
                'Worth it po bilhin ang product will order again.' ,
                'Unsatisfied',
                'Nice quality, will order again.' ,
                'Poor quality',
                'sir okay armygreen shorts nice',
                'Ang pangit ng quality! wag kayo bumili dito']

print(fake_reviews)

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

['Ang panget!', 'Hindi na ko bibili ulit, kulang ang pinadala niyo!', 'Sobrang ganda po, salamat', 'Maliliit sizes, hindi kasya sakin kahit large na inorder ko.', 'Ang ganda po ng quality', 'Worth it po bilhin ang product will order again.', 'Unsatisfied', 'Nice quality, will order again.', 'Poor quality', 'sir okay armygreen shorts nice', 'Ang pangit ng quality! wag kayo bumili dito']
1/1 [==============================] - 0s 18ms/step
Ang panget!
[0.32920012]


Hindi na ko bibili ulit, kulang ang pinadala niyo!
[0.23684889]


Sobrang ganda po, salamat
[0.9896438]


Maliliit sizes, hindi kasya sakin kahit large na inorder ko.
[0.16591594]


Ang ganda po ng quality
[0.95122135]


Worth it po bilhin ang product will order again.
[0.9500952]


Unsatisfied
[0.37267974]


Nice quality, will order again.
[0.9512565]


Poor quality
[0.15695202]


sir okay armygreen shorts nice
[0.9711775]


Ang pangit ng quality! wag kayo bumili dito
[0.27839127]


